<a href="https://colab.research.google.com/github/junhyeokk/boomhill24/blob/main/data/kart_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
from tqdm.notebook import tqdm

In [ ]:
def mse(img): # Error between input image and white image
  return np.sum((255-img)**2)/(np.shape(img)[0]*np.shape(img)[1])
  
def is_pushed(img): # image is pushed image if error is lager than threshold(200) (check image is almost white)
                    # when not pushed, image is almost white
  return 1 if mse(img) > 200 else 0

In [ ]:
# video 파일 이름
vidname = "kart_test"
# 카트가 사용하는 엔진 입력
enginetype = 'x'

In [ ]:
# 저장 path 설정
drivepath = "/content/gdrive/MyDrive/"
capdir1 = f"{vidname}_screen_mark3_4/"             # 이미지 저장 directory 이름
capdir2 = f"{vidname}_minimap_mark3_4"
csvname = f"{vidname}_minimap_mark3_4.csv"                # csv 파일 이름
cappath1 = os.path.join(drivepath, capdir1)  # 이미지 저장 경로
cappath2 = os.path.join(drivepath, capdir2)
csvpath = os.path.join(drivepath, "csv/")  # csv 파일 경로
vidpath = os.path.join(drivepath, vidname + ".mp4")

# directory 없을 시 생성
if not os.path.exists(cappath1):
  os.makedirs(cappath1)

if not os.path.exists(cappath2):
  os.makedirs(cappath2)

if not os.path.exists(csvpath):
  os.makedirs(csvpath)

In [ ]:
# csv 파일 생성, 비디오 파일 open
cap = cv2.VideoCapture(vidpath)
fcsv = open(os.path.join(csvpath+csvname),'w',encoding='utf-8')
wrcsv = csv.writer(fcsv)

In [ ]:
# top left position of key
# up, left, right, ctrl, shift, down 
key_pos = [(1265,914), (1188,992), (1343,992), (1092,992), (1092,914), (1265,992)]

# 키 인식 이미지 크기
key_size = 20

# 계기판 숫자 위치 (V1 엔진 or X 엔진)
# 3자리 일 경우
if enginetype == 'v1':
  gauge100pos = [649,965]
  gauge010pos = [697,965]
  gauge001pos = [745,965]
  # 2자리 일 경우
  gauge10pos = [673,965]
  gauge01pos = [721,965]
  gauge_w = 48; gauge_h = 56
if enginetype == 'x':
  gauge100pos = [672,982]
  gauge010pos = [703,982]
  gauge001pos = [734,982]
  # 2자리 일 경우
  gauge10pos = [688,982]
  gauge01pos = [719,982]
  gauge_w = 31; gauge_h = 56

prevpath = ""
prevminimap = ""
prevspeed = 0

In [ ]:
gauge = []
# 저장된 계기판 숫자 예시 이미지 로드
# x 엔진 : /xnum/  v1 엔진 : /v1num/
numpath = os.path.join(drivepath,enginetype+'num/')
print(numpath)
for i in np.arange(10):
  gaugepath = os.path.join(numpath + f'/num{i}.png')
  gaugenum = cv2.imread(gaugepath, cv2.IMREAD_GRAYSCALE)
  _, num_thresh = cv2.threshold(gaugenum, 135, 255, cv2.THRESH_BINARY)
  gauge.append(num_thresh)

/content/gdrive/MyDrive/xnum/


In [ ]:
# 숫자 이미지를 예시 이미지와 매칭
def gauge_match(numimg):
  _, img_thresh = cv2.threshold(numimg, 135, 255, cv2.THRESH_BINARY)
  for i, num in enumerate(gauge):
    diff = cv2.bitwise_xor(img_thresh,num)
    diffcount = cv2.countNonZero(diff)
    #plt.subplot(2,5,i+1)
    #plt.imshow(diff,cmap='gray')
    if diffcount < 200:
      #plt.show()
      #print(f'{i}th diff is {diffcount}')
      return i
  # 매칭하는 이미지가 없다면 0
  #plt.show()
  return 0

In [ ]:
def gauge_speed(frame):
  # 현재 계기판 이미지 자리별 추출 (자리가 매치하지 않으면 0로 계산)
  gaugeimg_100 = cv2.cvtColor(frame[gauge100pos[1]:gauge100pos[1]+gauge_h,
                                    gauge100pos[0]:gauge100pos[0]+gauge_w],
                              cv2.COLOR_BGR2GRAY)
  gaugeimg_010 = cv2.cvtColor(frame[gauge010pos[1]:gauge010pos[1]+gauge_h,
                                    gauge010pos[0]:gauge010pos[0]+gauge_w],
                              cv2.COLOR_BGR2GRAY)
  gaugeimg_001 = cv2.cvtColor(frame[gauge001pos[1]:gauge001pos[1]+gauge_h,
                                    gauge001pos[0]:gauge001pos[0]+gauge_w],
                              cv2.COLOR_BGR2GRAY)  
  gaugeimg_10 = cv2.cvtColor(frame[gauge10pos[1]:gauge10pos[1]+gauge_h,
                                   gauge10pos[0]:gauge10pos[0]+gauge_w],
                             cv2.COLOR_BGR2GRAY)
  gaugeimg_01 = cv2.cvtColor(frame[gauge01pos[1]:gauge01pos[1]+gauge_h,
                                   gauge01pos[0]:gauge01pos[0]+gauge_w],
                              cv2.COLOR_BGR2GRAY)
  speed_100 = gauge_match(gaugeimg_100)
  speed_010 = gauge_match(gaugeimg_010)
  speed_001 = gauge_match(gaugeimg_001)
  speed_10 = gauge_match(gaugeimg_10)
  speed_01 = gauge_match(gaugeimg_01)

  
  # 자리가 매치하지 않으면 0으로 계산됨(무시됨)    
  return speed_100*100 + speed_010*10 + speed_001 + speed_10*10 + speed_01

영상 추출 관련 기본 데이터

In [ ]:
fps = round(cap.get(cv2.CAP_PROP_FPS))
start_sec = 1404        # Start time
end_sec = 1511         # End time
interval = 0.1          # Time interval
# 단위는 초

print(f"fps : {fps}")

fps : 60


In [ ]:
for t in tqdm(np.arange(start_sec, end_sec, interval)):
  cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
  ret,frame = cap.read()

  if not ret:
    print("영상 시간 초과 또는 영상 관련 오류")


  '''
  speed = gauge_speed(frame)
  
  # 계기판 이미지 출력 후 비교
  gaugeimg_total = cv2.cvtColor(frame[int(gauge100pos[1]):int(gauge100pos[1]+gauge_h),
                                   int(gauge100pos[0]):int(gauge001pos[0]+gauge_w)],
                              cv2.COLOR_BGR2GRAY)
  _, gauge_thresh = cv2.threshold(gaugeimg_total, 135, 255, cv2.THRESH_BINARY)
  plt.imshow(gauge_thresh,cmap='gray')
  plt.title(speed)
  plt.show()
  '''



  speed = gauge_speed(frame)

  # 키 입력 추출
  key_array = [is_pushed(frame[index[1]:index[1]+key_size,index[0]:index[0]+key_size]) for index in key_pos]

  # 추출된 키 입력을 string type 변경
  keystring = ''.join(str(key) for key in key_array)
  #print(f"key string at {t} is {keystring}")

  # 게임 화면만 저장
  cropped_img = frame[:1080,:1440]
  minimap_img = frame[401:637, 1164:1416]

  # 이미지 파일에서 키 뷰어부분 마스킹
  frame[881:1080,1056:1440] = 0

  # 이미지 저장 경로
  imgpath1 = os.path.join(capdir1, f"t{t}.jpg")
  imgpath2 = os.path.join(capdir2, f"t{t}.jpg")

  # 이미지 저장
  cv2.imwrite(os.path.join(drivepath, imgpath1), cropped_img)
  cv2.imwrite(os.path.join(drivepath, imgpath2), minimap_img)

  # csv 파일 저장 (이전 이미지 저장 경로, 이전 이미지 속도, 입력)
  wrcsv.writerow([prevpath, prevminimap, prevspeed, keystring])

  t += interval
  prevpath = imgpath1
  prevminimap = imgpath2
  prevspeed = speed



In [ ]:
cap.release()
fcsv.close()